In [88]:
import numpy as np
import pandas as pd
import sqlite3
from datetime import date

pd.set_option('display.max_colwidth', None)

In [89]:
SQLITE_DB_PATH = 'redfin_scraper_data.db'
conn = sqlite3.connect(SQLITE_DB_PATH)
cur = conn.cursor()

In [102]:
cur.execute("PRAGMA table_info(listing_full_details)")
table_info = cur.fetchall()

today = date.today().strftime('%Y/%m/%d')
prev_day = '2022/02/11'
select_batch_query = f"SELECT * FROM listing_full_details WHERE STATUS='Active' AND DATE='{today}' AND NUMBER_ROOMS>=3 AND NUMBER_BATHROOMS>=1.5"
print(select_batch_query)
cur.execute(select_batch_query)
query_results = cur.fetchall()

SELECT * FROM listing_full_details WHERE STATUS='Active' AND DATE='2022/02/13' AND NUMBER_ROOMS>=3 AND NUMBER_BATHROOMS>=1.5


In [103]:
query_results

[]

In [19]:
column_names = [info[1] for info in table_info]

In [74]:
df = pd.DataFrame(query_results, columns=column_names)

In [75]:
standardizaiton_columns = ['PRICE', 'MORTGAGE', 'YEAR', 'SQFT', 'LOT_SIZE', 'SQFT_PRICE']

lot_size_weight = 0.8
for c in standardizaiton_columns:
    col_range = df[c].max() - df[c].min()
    col_min = df[c].min()
    if c == 'PRICE' or c == 'MORTGAGE':
        df['STD_BATCH_' + c] = df.apply(lambda row: 1 - ((row[c] - col_min) / col_range), axis=1)
    elif c == 'LOT_SIZE':
        df['STD_BATCH_' + c] = df.apply(lambda row: ((row[c] - col_min) / col_range) * lot_size_weight, axis=1)
    elif c == 'SQFT_PRICE':
        new_col = 'SQFT_PER_THOUSAND'
        df[new_col] = df.apply(lambda row: row['SQFT'] / (row['PRICE'] / 1000), axis=1)
        col_range = df[new_col].max() - df[new_col].min()
        col_min = df[new_col].min()
        df['STD_BATCH_' + new_col] = df.apply(lambda row: (row[new_col] - col_min) / col_range, axis=1)
    else:
        df['STD_BATCH_' + c] = df.apply(lambda row: (row[c] - col_min) / col_range, axis=1)

In [78]:
df[['STD_BATCH_PRICE', 
    'STD_BATCH_MORTGAGE', 
    'STD_BATCH_YEAR', 
    'STD_BATCH_SQFT', 
    'STD_BATCH_LOT_SIZE',
    'STD_BATCH_SQFT_PER_THOUSAND']].describe()

,STD_BATCH_PRICE,STD_BATCH_MORTGAGE,STD_BATCH_YEAR,STD_BATCH_SQFT,STD_BATCH_LOT_SIZE,STD_BATCH_SQFT_PER_THOUSAND
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,0.420735,0.415930,0.839728,0.325942,0.312643,0.309800
std,0.274542,0.236550,0.356787,0.168124,0.238475,0.172381
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.208935,0.208924,0.977992,0.184797,0.000000,0.190682
50%,0.368342,0.417847,0.988625,0.299694,0.333346,0.298903
75%,0.650324,0.606586,0.999011,0.444954,0.483337,0.419874
max,1.000000,1.000000,1.000000,1.000000,0.800000,1.000000


In [80]:
df['SCORE'] = df.apply(lambda row: np.sum([row['STD_BATCH_PRICE'], 
                                           row['STD_BATCH_MORTGAGE'], 
                                           row['STD_BATCH_YEAR'], 
                                           row['STD_BATCH_SQFT'], 
                                           row['STD_BATCH_LOT_SIZE'],
                                           row['STD_BATCH_SQFT_PER_THOUSAND']]), axis=1)

In [81]:
df['SCORE'].describe()

count    99.000000
mean      2.624777
std       0.824663
min       0.847629
25%       2.253853
50%       2.851145
75%       3.193266
max       4.077647
Name: SCORE, dtype: float64

In [86]:
df_sorted = df.sort_values(by=['SCORE'], ascending=False)
df_sorted.head(10)

,URL,DATE,STATUS,PRICE,NUMBER_ROOMS,NUMBER_BATHROOMS,SQFT,TIME_ON_REDFIN,YEAR,LOT_SIZE,...,SQFT_PRICE,MORTGAGE,STD_BATCH_PRICE,STD_BATCH_MORTGAGE,STD_BATCH_YEAR,STD_BATCH_SQFT,STD_BATCH_LOT_SIZE,SQFT_PER_THOUSAND,STD_BATCH_SQFT_PER_THOUSAND,SCORE
21,https://redfin.com/WA/Vancouver/9004-NE-73rd-St-98662/home/14592617,2022/02/11,Active,475000,4,2.5,2547,2,2004,5227.0,...,186,2222,0.621702,0.543201,0.991098,0.636959,0.400000,5.362105,0.884686,4.077647
10,https://redfin.com/WA/Vancouver/3117-NE-135th-Ave-98682/home/14673804,2022/02/11,Active,450000,3,2.5,1884,22,1979,10454.0,...,239,2045,0.746142,0.668555,0.978734,0.347313,0.800000,4.186667,0.512608,4.053353
95,https://redfin.com/WA/Vancouver/15215-SE-1st-St-98684/home/14611592,2022/02/11,Active,400000,3,2.0,1290,13,1978,9147.0,...,310,1577,0.995022,1.000000,0.978239,0.087811,0.699981,3.225000,0.208198,3.969252
9,https://redfin.com/WA/Vancouver/1417-NW-93rd-St-98665/home/14675850,2022/02/11,Active,449900,3,2.5,1750,4,1971,8712.0,...,257,1788,0.746640,0.850567,0.974777,0.288772,0.666692,3.889753,0.418622,3.946071
8,https://redfin.com/WA/Vancouver/5606-NE-66th-Cir-98661/home/14655205,2022/02/11,Active,445000,3,2.5,1680,29,2006,8428.0,...,265,1832,0.771030,0.819405,0.992087,0.258191,0.644959,3.775281,0.382386,3.868059
0,https://redfin.com/WA/Vancouver/9312-NE-80th-Ave-98662/home/14615966,2022/02/11,Active,425000,3,2.0,1563,13,1983,8276.0,...,272,1917,0.870582,0.759207,0.980712,0.207077,0.633327,3.677647,0.351481,3.802387
97,https://redfin.com/WA/Vancouver/5507-NE-Issler-St-98661/home/14599835,2022/02/11,Active,415000,4,2.0,1430,32,1974,8786.0,...,290,1897,0.920358,0.773371,0.976261,0.148973,0.672355,3.445783,0.278086,3.769405
6,https://redfin.com/WA/Vancouver/2610-NE-88th-Pl-98662/home/18425780,2022/02/11,Active,435000,3,2.5,1729,67,2008,3920.0,...,252,2015,0.820806,0.689802,0.993076,0.279598,0.299981,3.974713,0.445515,3.528779
3,https://redfin.com/WA/Vancouver/17728-SE-17th-Way-98683/home/14671031,2022/02/11,Active,430000,3,2.5,1534,4,1999,5662.0,...,280,2026,0.845694,0.682011,0.988625,0.194408,0.433289,3.567442,0.316596,3.460624
14,https://redfin.com/WA/Vancouver/7801-NE-67th-St-98662/home/14625139,2022/02/11,Active,460000,3,2.0,1710,3,1999,6534.0,...,269,2123,0.696366,0.613314,0.988625,0.271298,0.500019,3.717391,0.364062,3.433684


In [104]:
report_columns = ['URL', 'DATE', 'PRICE', 'MORTGAGE', 'YEAR', 'NUMBER_ROOMS', 'NUMBER_BATHROOMS', 'SQFT', 'LOT_SIZE', 'SQFT_PRICE', 'TIME_ON_REDFIN', 'SCORE']

In [107]:
df_sorted[report_columns].iloc[:5]

,URL,DATE,PRICE,MORTGAGE,YEAR,NUMBER_ROOMS,NUMBER_BATHROOMS,SQFT,LOT_SIZE,SQFT_PRICE,TIME_ON_REDFIN,SCORE
21,https://redfin.com/WA/Vancouver/9004-NE-73rd-St-98662/home/14592617,2022/02/11,475000,2222,2004,4,2.5,2547,5227.0,186,2,4.077647
10,https://redfin.com/WA/Vancouver/3117-NE-135th-Ave-98682/home/14673804,2022/02/11,450000,2045,1979,3,2.5,1884,10454.0,239,22,4.053353
95,https://redfin.com/WA/Vancouver/15215-SE-1st-St-98684/home/14611592,2022/02/11,400000,1577,1978,3,2.0,1290,9147.0,310,13,3.969252
9,https://redfin.com/WA/Vancouver/1417-NW-93rd-St-98665/home/14675850,2022/02/11,449900,1788,1971,3,2.5,1750,8712.0,257,4,3.946071
8,https://redfin.com/WA/Vancouver/5606-NE-66th-Cir-98661/home/14655205,2022/02/11,445000,1832,2006,3,2.5,1680,8428.0,265,29,3.868059


In [108]:
df_sorted.iloc[:5]['URL']

21      https://redfin.com/WA/Vancouver/9004-NE-73rd-St-98662/home/14592617
10    https://redfin.com/WA/Vancouver/3117-NE-135th-Ave-98682/home/14673804
95      https://redfin.com/WA/Vancouver/15215-SE-1st-St-98684/home/14611592
9       https://redfin.com/WA/Vancouver/1417-NW-93rd-St-98665/home/14675850
8      https://redfin.com/WA/Vancouver/5606-NE-66th-Cir-98661/home/14655205
Name: URL, dtype: object